In [2]:
from transformers import RobertaTokenizer
from datasets import load_dataset
import os

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')

In [4]:
DATASET_PATH = "../dataset/codesearchnet-java-discovered/"

data_files = {"train": [], "valid": [], "test": []}

for file in os.listdir(DATASET_PATH):
    file_path = os.path.join(DATASET_PATH, file)
    if "train" in file:
        data_files["train"].append(file_path)
    elif "valid" in file:
        data_files["valid"].append(file_path)
    elif "test" in file:
        data_files["test"].append(file_path)

print(data_files)

dataset = load_dataset("json", data_files=data_files)
print(dataset)


{'train': ['../dataset/codesearchnet-java-discovered/java_train_0.jsonl', '../dataset/codesearchnet-java-discovered/java_train_1.jsonl', '../dataset/codesearchnet-java-discovered/java_train_6.jsonl', '../dataset/codesearchnet-java-discovered/java_train_8.jsonl', '../dataset/codesearchnet-java-discovered/java_train_5.jsonl', '../dataset/codesearchnet-java-discovered/java_train_11.jsonl', '../dataset/codesearchnet-java-discovered/java_train_13.jsonl', '../dataset/codesearchnet-java-discovered/java_train_15.jsonl', '../dataset/codesearchnet-java-discovered/java_train_7.jsonl', '../dataset/codesearchnet-java-discovered/java_train_3.jsonl', '../dataset/codesearchnet-java-discovered/java_train_2.jsonl', '../dataset/codesearchnet-java-discovered/java_train_12.jsonl', '../dataset/codesearchnet-java-discovered/java_train_4.jsonl', '../dataset/codesearchnet-java-discovered/java_train_9.jsonl', '../dataset/codesearchnet-java-discovered/java_train_14.jsonl', '../dataset/codesearchnet-java-discover

Using custom data configuration default-70a21001498a12cd
Found cached dataset json (/home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine'],
        num_rows: 454273
    })
    valid: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine'],
        num_rows: 15326
    })
    test: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine'],
        num_rows: 26902
    })
})


In [5]:
MAX_INPUT_LENGTH = 512

def preprocess_examples(examples):
    contents = examples["contents"]
    xmi = examples["xmi"]

    model_inputs = tokenizer(contents, padding="max_length")

    """ labels = tokenizer(xmi, padding="longest").input_ids
    # -100 is a special value that the loss function will ignore
    labels = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels
    ] """
    labels = tokenizer(xmi).input_ids

    model_inputs["labels"] = labels

    return model_inputs


tokenized_dataset = dataset.map(preprocess_examples, batched=True, batch_size=100, num_proc=64)


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-9b48c6ca494db65d.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-296b089cb10602b4.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-ceb56a55f482b960.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-e9ca8a521dc28b85.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-261498055b02e9ad.arrow


...

Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-bfd12f9fde650221.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-c166ded59a1bce89.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-799dcbdf5103d24d.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-933d4df957d4740c.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-b0d2f82b0362a40f.arrow


In [6]:
clean_dataset = tokenized_dataset.filter(lambda example: len(example["input_ids"]) <= 512, num_proc=64)

Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-802f457b1900cdf4_00000_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-802f457b1900cdf4_00001_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-802f457b1900cdf4_00002_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-802f457b1900cdf4_00003_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-802f457b1900cdf4_00004_of_00064.arrow


...

Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-aefe86664ba6e216_00059_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-aefe86664ba6e216_00060_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-aefe86664ba6e216_00061_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-aefe86664ba6e216_00062_of_00064.arrow


Loading cached processed dataset at /home/nicolasmaier/.cache/huggingface/datasets/json/default-70a21001498a12cd/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-aefe86664ba6e216_00063_of_00064.arrow


In [8]:
clean_dataset.save_to_disk("../dataset/hf_dataset")

Flattening the indices:   0%|          | 0/426 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/15 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/26 [00:00<?, ?ba/s]

In [16]:
pt_dataset = clean_dataset.with_format("torch")
print("a:", tokenizer.eos_token, " - ", tokenizer.eos_token_id)
print(tokenizer.decode(pt_dataset["train"][0]["input_ids"]))

a: </s>  -  2
<s>package targetpackage;

public class TargetClass {
    protected final void bindIndexed(ConfigurationPropertyName name, Bindable<?> target,
			AggregateElementBinder elementBinder, ResolvableType aggregateType,
			ResolvableType elementType, IndexedCollectionSupplier result) {
		for (ConfigurationPropertySource source : getContext().getSources()) {
			bindIndexed(source, name, target, elementBinder, result, aggregateType,
					elementType);
			if (result.wasSupplied() && result.get()!= null) {
				return;
			}
		}
	}
}
</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad